In [ ]:
import json
from sklearn.model_selection import train_test_split
f = open('train-v2.0.json')
data = json.load(f)

In [ ]:
import pandas as pd
df=pd.read_csv('train_data.csv')
df.head()

In [ ]:
json_file_path = "test.json"

with open(json_file_path, encoding="utf-8") as f:
    squad = json.load(f)

squad


In [ ]:
df_train ,df_dev=train_test_split(df, test_size=0.2, random_state=42,stratify=df['Answer_possible'])


In [ ]:
def process_data(df):
    dict_final={}
    for i in range(df.shape[0]):
        dat=df.iloc[i]
        dict_final[dat["Theme"]]={}
    
    for i in range(df.shape[0]):
        dat=df.iloc[i]
        dict_final[dat["Theme"]][dat["Paragraph"]]=[]

    for i in range(df.shape[0]):
        dat=df.iloc[i]
        tmp={'Question':dat['Question'],'Answer_possible':dat['Answer_possible'],
                    'Answer_text':dat['Answer_text'],'Answer_start':dat['Answer_start'],'id':dat['Unnamed: 0']}
        tmp['Answer_text']=tmp['Answer_text'][1:-1]
        tmp["Answer_text"]=[tmp["Answer_text"]]
        # print("Type Answer is ",type(tmp["Answer_text"]))
        # print("Answer is ",tmp["Answer_text"][0])
        # break
        tmp['Answer_start']=tmp['Answer_start'][1:-1].split(',')
        if tmp['Answer_possible']==True:
            for i in range(len(tmp['Answer_start'])):
                tmp['Answer_start'][i]=int(tmp['Answer_start'][i])
        else :
            tmp['Answer_start']=[]
            tmp['Answer_text']=[]
        dict_final[dat["Theme"]][dat["Paragraph"]].append(tmp)


    cnt=[]
    def clean(dict_final):
        for i in dict_final.keys():
            for j in dict_final[i].keys():
                for k in dict_final[i][j]:
                    if not(len(k['Answer_text'])==len(k['Answer_start'])):
                        cnt.append(1)
                        print(len(k['Answer_text'])," ",len(k['Answer_start']))
                        print()
        print(len(cnt))

    clean(dict_final)


    final_json={'version':0.0,'data':[]}
    for i in dict_final.keys():
        tmp={'title':i,'paragraphs':[]}
        for j in dict_final[i].keys():
            tp={'qas':[],'context':(j[1:-1] if j[0]=='\'' else j)}
            for k in dict_final[i][j]:
                ty={}
                ty['question']=k['Question']
                ty['is_impossible']=(False if (k['Answer_possible']==True) else True)
                if ty['is_impossible']==True:
                    ty['plausible_answers']=[]
                ty['id']=str(k['id'])
                ty['answers']=[]
                if not(len(k['Answer_text'])==len(k['Answer_start'])):
                    print("Non Same at i = ",i," Para = ",j," question = ",k)
                    print(len(k['Answer_text']),"            ",len(k['Answer_start']))
                for l in range(len(k['Answer_text'])):
                    rr={}
                    rr['text']=(k['Answer_text'][l][1:-1] if k['Answer_text'][l][0]=='\'' else k['Answer_text'][l])
                    rr['answer_start']=int(k['Answer_start'][l])
                    ty['answers'].append(rr)
                tp['qas'].append(ty)
            tmp['paragraphs'].append(tp)
        final_json['data'].append(tmp)

    print("**********************Processed DF ****************** ")

    return final_json

In [ ]:
import json
import numpy as np
def save_json(name,final_json):
    json_object = json.dumps(final_json)
    
    # Writing to sample.json
    with open(name, "w") as outfile:
        outfile.write(json_object)

In [ ]:
test=process_data(df_train)
dev=process_data(df_dev)

In [ ]:
save_json('test.json',test)
save_json('dev.json',dev)